<a href="https://colab.research.google.com/github/yueguo1997/SNA_recommender_system/blob/main/GNN_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
import os
import networkx as nx
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
cur_path = "/content/drive/My Drive/SNA final project/"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/SNA final project


In [10]:
G_train = nx.read_graphml("train.graphml")

# Node embedding

# Model developing

In [17]:
node_types = np.array([n[1]['type'] for n in G_train.nodes(data=True)])
edge_types = np.array([e[2]['type'] for e in G_train.edges(data=True)])
enc = OneHotEncoder(sparse=False)
node_types_enc = enc.fit_transform(node_types.reshape(-1, 1))
edge_types_enc = enc.fit_transform(edge_types.reshape(-1, 1))

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [26]:
node_types_enc

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [24]:
def GNN(input_dim, hidden_dim, output_dim):
    x_in = Input(shape=(input_dim,))
    a_in = Input(shape=(None,))
    h = Dense(hidden_dim, activation='relu')(x_in)
    idx = tf.cast(tf.where(tf.not_equal(a_in, 0)), dtype=tf.int32)
    updates = tf.gather(h, idx[:, 0])
    h = tf.tensor_scatter_nd_update(tf.zeros_like(h), idx, updates)
    h = Dense(output_dim, activation=None)(h)
    return Model(inputs=[x_in, a_in], outputs=h)

In [25]:
input_dim = node_types_enc.shape[1]
hidden_dim = 16
output_dim = 8
gnn = GNN(input_dim, hidden_dim, output_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
for epoch in range(100):
    with tf.GradientTape() as tape:
        node_embed = gnn([node_types_enc, np.array(nx.to_numpy_matrix(g, weight='edge_type', multigraph_weight=max))])
        loss = tf.reduce_mean(tf.square(node_embed))
    grads = tape.gradient(loss, gnn.trainable_variables)
    optimizer.apply_gradients(zip(grads, gnn.trainable_variables))
    print(f"Epoch {epoch+1} - Loss: {loss.numpy():.4f}")

# Get node embeddings
node_embed = gnn([node_types_enc, np.array(nx.to_numpy_matrix(g, weight='edge_type', multigraph_weight=max))])

ValueError: ignored